#Introduction

This report is the final project of A/B Testing Course of Udacity. The structure of the report follows the suggested structure of the Udacity for the project report, with some further explanations where ever it is needed.

The project is about an A/B Testing, not surprisingly, an experiment that Udacity, an e-learning platform, wants to do on the users of its courses.

More specifically, an experimental step would be added after the “Start Free Trial” button on the course page. Based on this experiment, after clicking on the “Start Free Trial” button, a message would be shown asking users the weekly time that they want to invest on the course. If the input hours is below 5 hours per week, “a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggestiong that the student might like to access the course materials for free.”

The goal of the experiment is reducing the number of students who would quit the course during the starting 14 days trial, while leaving the number of students who pass this 14 days period intact. 

#Experiment Design
The unit of diversion is a cookie, although if the student enrolls in the free trial, they are tracked by user-id from that point forward. The same user-id cannot enroll in the free trial twice. For users that do not enroll, their user-id is not tracked in the experiment, even if they were signed in when they visited the course overview page.

##Metric Choice
**Invariant Metrics** : number of cookies, number of clicks, click-through-probability.
**Evaluation Metrics** : gross conversion, retention, net conversion.

###**Evaluation Metrics**

Evaluation metrics are chosen since there is a possibility of different distribution between experiment and control groups as a function of experiment. Each evaluation metric is associated with a minimum difference (dmin) that must be observed for consideration in the decision to launch the experiment. The ultimate goal is to minimize student frustation and use the limited coaching resources most efficiently. With this in mind, the following conditions must be satisfied -

-Increased retension, i.e, the ratio of users who remained enrolled past the 14-day boundary to the number of users to complete checkout should increase.

-Decreased gross conversion coupled to increase in net conversion, i.e, less students enrolling in free trial but more students staying beyound the free trial.

**Gross conversion**: That is, number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button.

**Retention**: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to complete checkout.

**Net conversion**: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button.

##Sizing
###Number of Samples and Power
Page views required for each evaluation metric is calculated separately using the online calculator. The alpha vaue of 0.05 and beta value of 0.2 is used in all the cases.

***Pageviews for Each Evaluation Metric to Achieve Target Statistical Power***

**Gross Conversion**
- Baseline Conversion: 20.625%
- Minimum Detectable Effect: 1%
- alpha: 5%
- beta: 20%
- 1 - beta: 80%
- sample size = 25,835 enrollments/group
- Number of groups = 2 (experiment and control)
- total sample size = 51,670 enrollments
- clicks/pageview: 3200/40000 = .08 clicks/pageview
- pageviews = 645,875

**Retention**
- Baseline Conversion: 53%
- Minimum Detectable Effect: 1%
- alpha: 5%
- beta: 20%
- 1 - beta: 80%
- sample size = 39,155 enrollments/group
- Number of groups = 2 (experiment and control)
- total sample size = 78,230 enrollments
- enrollments/pageview: 660/40000 = .0165 enrollments/pageview
- pageviews = 78,230/.0165 = 4,741,212

**Net Conversion**
- Baseline Conversion: 10.9313%
- Minimum Detectable Effect: .75%
- alpha: 5%
- beta: 20%
- 1 - beta: 80%
- sample size = 27,413 enrollments/group
- Number of groups = 2 (experiment and control)
- total sample size = 54,826
- clicks/pageview: 3200/40000 = .08 clicks/pageview
- pageviews = 685,325

###Duration and Exposure

In [1]:
4741212.0/40000

118.5303

In [2]:
685325.0/40000

17.133125

If we divert 100% off traffic, given 40,000 page views per day, the experiment would take 119 days. That is a long time. If we eliminate retention, we are left with Gross Conversion and Net Conversion. This reduces the number of required pageviews to 685,325, and an 18 day experiment with 100% diversion. There may be other experiments to run, so let's say 50% diversion for 35 days.

In [3]:
import pandas as pd
import numpy as np

In [4]:
df_control = pd.read_csv("C:/Users/siyue/OneDrive/Desktop/ab project/Final Project Results - Control.csv")
df_experiment = pd.read_csv("C:/Users/siyue/OneDrive/Desktop/ab project/Final Project Results - Experiment.csv")

In [5]:
df_control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [6]:
df_experiment.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [11]:
results = {"Control":pd.Series([df_control.Pageviews.sum(),df_control.Clicks.sum(),
                                  ],
                                  index = ["cookies","clicks"]),
           "Experiment":pd.Series([df_experiment.Pageviews.sum(),df_experiment.Clicks.sum(),
                                 ],
                               index = ["cookies","clicks"])}
df_results = pd.DataFrame(results)
df_results

,Control,Experiment
cookies,345543,344660
clicks,28378,28325


###Sanity Check
**Count Metrics

In [12]:
df_results['Total']=df_results.Control + df_results.Experiment
df_results['Prob'] = 0.5
df_results['StdErr'] = np.sqrt((df_results.Prob * (1- df_results.Prob))/df_results.Total)
df_results["MargErr"] = 1.96 * df_results.StdErr
df_results["CI_lower"] = df_results.Prob - df_results.MargErr
df_results["CI_upper"] = df_results.Prob + df_results.MargErr
df_results["Obs_val"] = df_results.Experiment/df_results.Total
df_results["Pass_Sanity"] = df_results.apply(lambda x: (x.Obs_val > x.CI_lower) and (x.Obs_val < x.CI_upper),axis=1)
df_results['Diff'] = abs((df_results.Experiment - df_results.Control)/df_results.Total)

df_results

,Control,Experiment,Total,Prob,StdErr,MargErr,CI_lower,CI_upper,Obs_val,Pass_Sanity,Diff
cookies,345543,344660,690203,0.5,0.000602,0.001180,0.498820,0.501180,0.499360,True,0.001279
clicks,28378,28325,56703,0.5,0.002100,0.004116,0.495884,0.504116,0.499533,True,0.000935


**Other Metrics**

In [9]:
# click through probability (clicks/cookies)

control_cookies = df_results.loc['cookies','Control']
control_clicks = df_results.loc['clicks','Control']

exp_cookies = df_results.loc['cookies','Experiment']
exp_clicks = df_results.loc['clicks', 'Experiment']

## control value 
cont_p_hat = control_clicks/control_cookies

## observed value (experimental value)
exp_p_hat = exp_clicks/exp_cookies

## Standard Error
SE_ClickProb = np.sqrt((cont_p_hat * (1- cont_p_hat))/control_cookies)


## margin of error for 95% confidence interval (z = 1.96)

ME_ClickProb = SE_ClickProb * 1.96

## CI
upper_ClickProb = exp_p_hat + ME_ClickProb
lower_ClickProb = exp_p_hat - ME_ClickProb

## Sane in the membrane (yes, it passes)
print(cont_p_hat,exp_p_hat,lower_ClickProb,upper_ClickProb, SE_ClickProb, ME_ClickProb)

0.08212581357457682 0.08218244066616376 0.08126698684411665 0.08309789448821087 0.0004670682765546443 0.0009154538220471028


It passes.

###Check for Practical and Statistical Significance

In [13]:
df_control_notnull = df_control[pd.isnull(df_control.Enrollments) != True]
df_experiment_notnull = df_experiment[pd.isnull(df_control.Enrollments) != True]

In [14]:
results_notnull = {"Control":pd.Series([df_control_notnull.Pageviews.sum(),df_control_notnull.Clicks.sum(),
                                  df_control_notnull.Enrollments.sum(),df_control_notnull.Payments.sum()],
                                  index = ["cookies","clicks","enrollments","payments"]),
           "Experiment":pd.Series([df_experiment_notnull.Pageviews.sum(),df_experiment_notnull.Clicks.sum(),
                               df_experiment_notnull.Enrollments.sum(),df_experiment_notnull.Payments.sum()],
                               index = ["cookies","clicks","enrollments","payments"])}
df_results_notnull = pd.DataFrame(results_notnull)
df_results_notnull

,Control,Experiment
cookies,212163.0,211362.0
clicks,17293.0,17260.0
enrollments,3785.0,3423.0
payments,2033.0,1945.0


In [15]:
# experiment values

enrollments_exp = df_results_notnull.loc["enrollments"].Experiment
clicks_exp = df_results_notnull.loc["clicks"].Experiment
payments_exp = df_results_notnull.loc["payments"].Experiment

# control values

enrollments_cont = df_results_notnull.loc["enrollments"].Control
clicks_cont = df_results_notnull.loc["clicks"].Control
payments_cont = df_results_notnull.loc["payments"].Control



# metrics

GrossConversion_exp = enrollments_exp/clicks_exp
NetConversion_exp = payments_exp/clicks_exp
GrossConversion_cont = enrollments_cont/clicks_cont
NetConversion_cont = payments_cont/clicks_cont

GrossConversion = (enrollments_exp + enrollments_cont)/(clicks_cont + clicks_exp)
NetConversion = (payments_cont + payments_exp)/(clicks_cont + clicks_exp)

In [16]:
print('GrossConversion: {} \nNetConversion:{}'.format(GrossConversion,NetConversion))

GrossConversion: 0.20860706740369866 
NetConversion:0.1151274853124186


In [17]:
GrossConversion_cont

0.2188746891805933

In [18]:
GrossConversion_exp

0.19831981460023174

In [19]:
def stats_prop(p_hat,z_score,N_cont,N_exp,diff):
    std_err = np.sqrt((p_hat * (1- p_hat ))*(1/N_cont + 1/N_exp))
    marg_err = z_score * std_err
    ci_lower = diff - marg_err
    ci_upper = diff + marg_err
    
    return std_err,marg_err,ci_lower,ci_upper

In [20]:
GrossConversion_diff = GrossConversion_exp - GrossConversion_cont
GrossConversion_diff

-0.020554874580361565

In [22]:
se_gross,me_gross,cil_gross,ciu_gross = stats_prop(GrossConversion,1.96,clicks_cont,
                                                   clicks_exp,GrossConversion_diff)
print(se_gross,me_gross,cil_gross,ciu_gross)

0.004371675385225936 0.008568483755042836 -0.0291233583354044 -0.01198639082531873


As it can be seen, the 0 is out of CI boundary, and it shows that the obsevered result is statistically significant. Also since the result is on the left hand side of the 0, our expectation is met and the gross conversion rate of the experiment group is lower than the control group.

In [23]:
NetConversion_diff = NetConversion_exp - NetConversion_cont
NetConversion_diff

-0.0048737226745441675

In [24]:
se_net,me_net,cil_net,ciu_net = stats_prop(NetConversion,1.96,clicks_cont,
                                           clicks_exp,NetConversion_diff)
print(se_net,me_net,cil_net,ciu_net)

0.0034341335129324238 0.0067309016853475505 -0.011604624359891718 0.001857179010803383


As it can be seen, the difference between net conversion rate of the two expriment and control groups is insignificant. 

##Sign Test

In [25]:
df_SignTest = pd.merge(df_control_notnull,df_experiment_notnull,on="Date")
df_SignTest['GrossConversion_cont'] = df_SignTest.Enrollments_x/df_SignTest.Clicks_x
df_SignTest['GrossConversion_exp'] = df_SignTest.Enrollments_y/df_SignTest.Clicks_y
df_SignTest['NetConversion_cont'] = df_SignTest.Payments_x/df_SignTest.Clicks_x
df_SignTest['NetConversion_exp'] = df_SignTest.Payments_y/df_SignTest.Clicks_y

cols = ['Date','GrossConversion_cont','GrossConversion_exp','NetConversion_cont','NetConversion_exp']

In [26]:
df_SignTest = df_SignTest[cols]
df_SignTest.head()

,Date,GrossConversion_cont,GrossConversion_exp,NetConversion_cont,NetConversion_exp
0,"Sat, Oct 11",0.195051,0.153061,0.101892,0.049563
1,"Sun, Oct 12",0.188703,0.147771,0.089859,0.115924
2,"Mon, Oct 13",0.183718,0.164027,0.104510,0.089367
3,"Tue, Oct 14",0.186603,0.166868,0.125598,0.111245
4,"Wed, Oct 15",0.194743,0.168269,0.076464,0.112981


In [27]:
df_SignTest['GC_Sign'] = df_SignTest.GrossConversion_cont - df_SignTest.GrossConversion_exp
df_SignTest['NC_Sign'] = df_SignTest.NetConversion_cont - df_SignTest.NetConversion_exp

len(df_SignTest)

23

In [31]:
len(df_SignTest[df_SignTest.GC_Sign > 0])

19

In [32]:
len(df_SignTest[df_SignTest.NC_Sign < 0])

10

**Gross conversion**

- Number of success: 19, which is the number of days that we see a decrease in the number of enrollments in the experimental group.
- Number of trials: 23, which is the toal 23 days of the experiments.
- Probability: 0.5
- Two-tailed p-value: 0.0026

Since the two-tailed p-value is smaller than alpha = 0.05, the change is statistically significant.

**Net conversion**

- Number of success: 10, which is the number of days that we see an improvement in the number of payments in the experimental group.
- Number of trials: 23, which is the toal 23 days of the experiments.
- Probability: 0.5
- Two-tailed p-value: 0.6776

Since the two-tailed p-value is larger than alpha = 0.05, the change is not statistically significant.

##Recommendation

My recommendation is to **not** implement this new feature into the overview page of Udacity. As a reminder, the original goal for adding this new feature is to decrease in gross conversion and no decrease in net conversion. The result shows that the gross conversion turned out to be negative and practically significant, which is a good result for this metric and for Udacity. This is because it would lower the cost as well as time for the Udacity team to take care of these users who would not have time to complete the courses and thus unlikely to convert. However, the result also shows the confidence interval of the net conversion does include negative numbers of the practical boundary, which implies it is possible that this evaluation metrics got decreased by an amount that could potentially hurt the business. In other words, adding the time commitment alert feature would not help to increase the number of paid users, and hence the revenue of the company.